# Problem 5 - Personalized Assortment Optimization at Birchbox

In [1]:
import pandas as pd
import os
from pyomo.environ import *
from pyomo.opt import SolverFactory
os.chdir(r"C:\Users\User\Desktop\BUS 730 - Prescriptive Modeling & Optimization for BA\Assignments\HW3_Integer (MIP) Optimization")
%pprint

Pretty printing has been turned OFF


## Part 1. The Base Integer Program Formulation

In [2]:
# reading in item info data 
itemdata = pd.read_excel(open('Birchboxdata2020.xlsx','rb'), sheet_name='item info') 
itemdata

,Item,Category,Inventory,Sample Cost,Margin if Purchase,PurchaseProb_Seg_0,PurchaseProb_Seg_1,PurchaseProb_Seg_2,PurchaseProb_Seg_3,PurchaseProb_Seg_4,PurchaseProb_Seg_5,PurchaseProb_Seg_6,PurchaseProb_Seg_7,PurchaseProb_Seg_8,PurchaseProb_Seg_9,PurchaseProb_Seg_10,PurchaseProb_Seg_11,PurchaseProb_Seg_12,PurchaseProb_Seg_13
0,0,6,94,7.5,51.475382,0.090036,0.100748,0.248534,0.288539,0.104476,0.112507,0.091812,0.120041,0.087837,0.089800,0.098098,0.092363,0.113431,0.115416
1,1,7,107,3.2,26.316932,0.288539,0.093838,0.122771,0.088250,0.100276,0.100276,0.096180,0.109183,0.094151,0.101234,0.092085,0.103327,0.092646,0.099370
2,2,3,119,0.8,3.772008,0.098511,0.248534,0.086859,0.109183,0.101734,0.097696,0.107713,0.106349,0.092363,0.090036,0.124267,0.087636,0.098511,0.088675
3,3,2,153,9.5,50.397349,0.101734,0.087636,0.091545,0.088250,0.086859,0.116483,0.087049,0.182048,0.092646,0.147708,0.205559,0.088042,0.087837,0.182048
4,4,7,176,1.0,5.267752,0.100276,0.092935,0.099817,0.092085,0.173718,0.131383,0.094798,0.182048,0.087437,0.107713,0.088042,0.118790,0.093838,0.090770
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,106,1,180,6.4,25.380000,0.089124,0.082067,0.096791,0.106291,0.102378,0.113908,0.089246,0.096246,0.095338,0.095915,0.216858,0.095782,0.092276,0.111576
107,107,5,237,1.7,7.700000,0.169259,0.103963,0.098004,0.096389,0.109754,0.082806,0.156027,0.115600,0.100787,0.091678,0.127229,0.096544,0.086086,0.130835
108,108,5,144,4.2,15.180000,0.094417,0.084506,0.244216,0.300555,0.108563,0.106108,0.096553,0.118824,0.093697,0.105058,0.099148,0.092825,0.125323,0.119755
109,109,2,181,1.9,8.690000,0.296148,0.092574,0.117877,0.101158,0.102847,0.090533,0.111821,0.115647,0.088299,0.103798,0.090097,0.117044,0.082342,0.116514


In [3]:
# reading in customer info data 
custdata = pd.read_excel(open('Birchboxdata2020.xlsx','rb'), sheet_name='cust info') 
custdata

,Cust #,Seg,Item 0,Item 1,Item 2,Item 3,Item 4,Item 5,Item 6,Item 7,...,Item 101,Item 102,Item 103,Item 104,Item 105,Item 106,Item 107,Item 108,Item 109,Item 110
0,0,5,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,7,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,3,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,6,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1042,1042,3,0,0,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,1
1043,1043,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,1,0,0,0
1044,1044,10,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1045,1045,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1


In [4]:
# extract cust_item data for constraint use
cust_itemdf = custdata.loc[ : ,"Item 0":"Item 110"] 
cust_itemdf

,Item 0,Item 1,Item 2,Item 3,Item 4,Item 5,Item 6,Item 7,Item 8,Item 9,...,Item 101,Item 102,Item 103,Item 104,Item 105,Item 106,Item 107,Item 108,Item 109,Item 110
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
3,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,1,...,0,0,1,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1042,0,0,0,0,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,1
1043,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,1,0,0,0
1044,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1045,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1


In [5]:
# converting cust_itemdf to a list of lists
cust_item = cust_itemdf.values.tolist()
cust_item[0] # display the first list

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [6]:
# converting itemdata to a list of lists
iteminfo = itemdata.loc[:, 'Category':'PurchaseProb_Seg_13'].values.tolist()
iteminfo[0] # display the first list

[6.0, 94.0, 7.5, 51.47538228024059, 0.09003632671271142, 0.10074825950067987, 0.24853397382384473, 0.28853900817779266, 0.10447570262568968, 0.11250656562509134, 0.09181242444975067, 0.12004065140016523, 0.08783724495007438, 0.0897999130728838, 0.09809842472393217, 0.09236311002550751, 0.11343139682053335, 0.11541560327111706]

In [7]:
# converting segment to a list
segnum = custdata["Seg"].tolist()
segnum

[5, 4, 7, 0, 6, 4, 0, 9, 1, 12, 2, 13, 6, 12, 4, 1, 9, 5, 0, 9, 13, 9, 13, 12, 11, 5, 1, 7, 7, 5, 6, 7, 4, 13, 11, 4, 6, 3, 7, 3, 9, 5, 5, 12, 4, 10, 9, 2, 8, 13, 8, 8, 11, 3, 12, 4, 6, 13, 4, 9, 7, 11, 9, 13, 10, 2, 7, 2, 5, 5, 11, 4, 3, 13, 1, 13, 1, 3, 6, 5, 12, 6, 10, 5, 2, 13, 13, 1, 13, 8, 6, 2, 3, 12, 3, 8, 6, 3, 4, 6, 2, 5, 3, 5, 8, 5, 13, 13, 3, 10, 2, 1, 8, 7, 9, 4, 3, 13, 8, 7, 2, 10, 1, 5, 7, 2, 12, 13, 1, 4, 10, 13, 3, 5, 1, 9, 10, 11, 13, 4, 8, 6, 8, 1, 13, 9, 7, 5, 5, 10, 5, 12, 1, 3, 3, 8, 5, 2, 3, 10, 8, 12, 13, 6, 8, 5, 11, 4, 8, 13, 3, 10, 8, 11, 9, 11, 13, 3, 1, 9, 4, 8, 4, 12, 7, 3, 12, 3, 5, 11, 3, 13, 12, 5, 11, 3, 6, 10, 13, 8, 3, 5, 11, 2, 11, 3, 10, 1, 6, 12, 1, 2, 3, 3, 5, 7, 5, 12, 10, 9, 9, 9, 4, 12, 12, 8, 5, 10, 13, 3, 8, 12, 2, 5, 5, 5, 5, 9, 8, 5, 10, 7, 7, 9, 3, 8, 3, 1, 3, 1, 6, 6, 5, 5, 2, 5, 3, 1, 13, 5, 10, 12, 3, 8, 12, 9, 4, 2, 1, 9, 13, 3, 5, 2, 12, 11, 2, 5, 8, 7, 13, 6, 6, 10, 12, 1, 5, 10, 7, 1, 2, 6, 1, 3, 7, 10, 6, 9, 6, 12, 8, 4, 3, 7, 2, 

In [8]:
# declare a concrete model
model = ConcreteModel()

# declare the decision variables
n_cust = len(custdata)  # 1047
n_item = len(itemdata)  # 111

model.x = Var(range(n_cust), range(n_item), domain = Binary)  # Item sent to that customer? (n_cust*n_item)

# i refers to the customer
# j refers to the item

# specify the objective: Maximize the total expected margin minus the cost of the samples for the next round of boxes

# total expected margin: whether item sent * PurchaseProb_Seg of that item * margin if purchase
expectedmargin = sum(model.x[i,j]*iteminfo[j][segnum[i]+4]*iteminfo[j][3] for i in range(n_cust) for j in range(n_item))
# total sample costs: whether item sent * sample cost
samplecosts = sum(model.x[i,j]*iteminfo[j][2] for i in range(n_cust) for j in range(n_item))
model.Objective = Objective(expr = expectedmargin - samplecosts, sense= maximize) 

# Constraint: 5 items need to be sent to each customer
model.FiveItemSent =  ConstraintList() 
for i in range(n_cust): # for each customer
    model.FiveItemSent.add(expr = sum(model.x[i,j] for j in range(n_item)) == 5)

# Constraint: Can’t run out of inventory
model.InventoryLimit =  ConstraintList() 
for j in range(n_item): # for each item
    model.InventoryLimit.add(expr = sum(model.x[i,j] for i in range(n_cust)) <= iteminfo[j][1])

# Constraint: Can’t send a customer a sample they’ve already received
model.NeverRecieved =  ConstraintList() 
for i in range(n_cust): # for each customer
    model.NeverRecieved.add(expr = sum(model.x[i,j]*cust_item[i][j] for j in range(n_item)) == 0)

# specify the solver and solve
opt = SolverFactory('glpk')
results = opt.solve(model)#, tee = True)

In [9]:
# print the solution
def solutionprint():
    # Expected profit: total expected margin - the cost of the samples 
    print("Expected profit (total expected margin - sample costs):", model.Objective())

    # print the items sent to each customer
    print("\nThe items sent to each customer: \n")

    for i in range(n_cust):   # for each customer
        items = ""
        for j in range(n_item):  # # for each item
            if model.x[i,j]() != 0:
                items += '%-3s' % str(j) 
        print("Customer", i, ": send item ", items, "\n")   

solutionprint()  # obj: 15957.6570400054

Expected profit (total expected margin - sample costs): 15957.6570400054

The items sent to each customer: 

Customer 0 : send item  7  9  14 34 41  

Customer 1 : send item  24 51 67 94 105 

Customer 2 : send item  14 30 31 41 58  

Customer 3 : send item  12 22 28 55 109 

Customer 4 : send item  6  22 33 60 76  

Customer 5 : send item  24 67 78 94 105 

Customer 6 : send item  1  22 28 55 109 

Customer 7 : send item  16 23 50 61 104 

Customer 8 : send item  2  11 38 56 83  

Customer 9 : send item  12 16 43 70 98  

Customer 10 : send item  0  1  35 62 81  

Customer 11 : send item  21 22 30 71 98  

Customer 12 : send item  6  22 60 76 87  

Customer 13 : send item  12 16 43 70 98  

Customer 14 : send item  24 51 78 94 105 

Customer 15 : send item  6  11 38 56 83  

Customer 16 : send item  16 23 50 61 104 

Customer 17 : send item  7  9  14 34 41  

Customer 18 : send item  1  12 22 28 109 

Customer 19 : send item  16 23 50 61 104 

Customer 20 : send item  21 22 30 71 98  

Customer 363 : send item  0  8  35 62 81  

Customer 364 : send item  17 24 44 71 98  

Customer 365 : send item  14 30 41 58 98  

Customer 366 : send item  24 51 67 78 105 

Customer 367 : send item  24 51 67 94 105 

Customer 368 : send item  12 16 43 70 71  

Customer 369 : send item  24 51 78 94 105 

Customer 370 : send item  6  22 60 76 87  

Customer 371 : send item  17 24 44 71 98  

Customer 372 : send item  6  22 60 76 87  

Customer 373 : send item  0  19 27 81 100 

Customer 374 : send item  7  9  14 34 41  

Customer 375 : send item  14 17 44 71 98  

Customer 376 : send item  16 43 70 71 98  

Customer 377 : send item  14 31 41 58 98  

Customer 378 : send item  0  19 73 81 100 

Customer 379 : send item  0  19 27 81 100 

Customer 380 : send item  6  7  11 12 65  

Customer 381 : send item  21 22 30 71 98  

Customer 382 : send item  8  27 35 62 81  

Customer 383 : send item  0  19 27 81 100 

Customer 384 : send item  0  19 27 81 100 

Customer 385 : send item  17 21 

Customer 641 : send item  7  30 31 41 98  

Customer 642 : send item  16 23 50 58 104 

Customer 643 : send item  12 43 70 71 98  

Customer 644 : send item  14 41 44 71 98  

Customer 645 : send item  14 17 44 71 98  

Customer 646 : send item  7  9  14 34 41  

Customer 647 : send item  0  27 73 81 100 

Customer 648 : send item  7  16 50 58 61  

Customer 649 : send item  3  12 30 39 52  

Customer 650 : send item  6  11 38 65 92  

Customer 651 : send item  2  6  11 56 83  

Customer 652 : send item  12 16 43 70 98  

Customer 653 : send item  6  22 60 76 87  

Customer 654 : send item  6  22 60 76 87  

Customer 655 : send item  2  6  11 29 83  

Customer 656 : send item  6  60 76 87 103 

Customer 657 : send item  6  22 60 76 87  

Customer 658 : send item  3  12 25 39 52  

Customer 659 : send item  16 23 50 61 104 

Customer 660 : send item  6  11 38 65 92  

Customer 661 : send item  6  22 60 76 103 

Customer 662 : send item  24 51 78 94 105 

Customer 663 : send item  14 24 

Customer 951 : send item  12 16 70 71 98  

Customer 952 : send item  17 21 22 71 103 

Customer 953 : send item  6  7  12 38 92  

Customer 954 : send item  24 51 78 94 105 

Customer 955 : send item  14 31 41 58 98  

Customer 956 : send item  21 22 30 71 98  

Customer 957 : send item  7  9  14 34 88  

Customer 958 : send item  24 51 78 94 105 

Customer 959 : send item  17 21 22 71 98  

Customer 960 : send item  8  35 62 81 89  

Customer 961 : send item  23 50 58 61 104 

Customer 962 : send item  6  22 60 76 87  

Customer 963 : send item  12 16 43 70 98  

Customer 964 : send item  3  12 25 30 52  

Customer 965 : send item  9  14 31 34 41  

Customer 966 : send item  2  11 29 56 83  

Customer 967 : send item  7  12 43 70 98  

Customer 968 : send item  7  9  14 34 41  

Customer 969 : send item  0  19 73 81 100 

Customer 970 : send item  24 51 78 94 105 

Customer 971 : send item  3  12 25 30 52  

Customer 972 : send item  7  9  14 34 41  

Customer 973 : send item  14 31 

In [10]:
# save the solution to a dataframe
def solutiontodf():
    colname = cust_itemdf.columns.values.tolist()
    colname.insert(0, 'Cust #')

    solution = []
    for i in range(n_cust):   # for each customer
        custOutput = []
        custOutput.append(i)
        for j in range(n_item):  # # for each item
            custOutput.append(model.x[i,j]())
        solution.append(custOutput)

    df_sol = pd.DataFrame(solution, columns = colname)
    return df_sol

df_sol = solutiontodf()
df_sol

,Cust #,Item 0,Item 1,Item 2,Item 3,Item 4,Item 5,Item 6,Item 7,Item 8,...,Item 101,Item 102,Item 103,Item 104,Item 105,Item 106,Item 107,Item 108,Item 109,Item 110
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1042,1042,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1043,1043,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1044,1044,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1045,1045,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
# Export df_sol to excel
df_sol.to_excel("P5_BirchboxSolutionPart1_Shih-Yuan_Wang.xlsx", index=False)

### Interpretation of the solution

The binary result shows whether the item should be sent to the customer. A "1" indicates that this item sample should be sent to the customer. For example, the sample of the item 7, 9, 14, 34, 41 should be send to Customer 0 this month.

The expected profit (total expected margin - sample costs) under the optimal solution: $ 15,957.65704

## Part 2. Next Month’s Revenue with Zero Replenishment

In [12]:
# Update the number of inventory remaining 

# The number of items sent this month
numitemsent = []
for j in range(n_item): # for each item
    numsent = sum(model.x[i,j]() for i in range(n_cust))
    numitemsent.append(numsent)

# The number of inventory remaining next month
for j in range(n_item): # for each item
    iteminfo[j][1] = iteminfo[j][1] - numitemsent[j]
iteminfo

[[6.0, 0.0, 7.5, 51.47538228024059, 0.09003632671271142, 0.10074825950067987, 0.24853397382384473, 0.28853900817779266, 0.10447570262568968, 0.11250656562509134, 0.09181242444975067, 0.12004065140016523, 0.08783724495007438, 0.0897999130728838, 0.09809842472393217, 0.09236311002550751, 0.11343139682053335, 0.11541560327111706], [7.0, 95.0, 3.2, 26.316931750100423, 0.28853900817779266, 0.09383768228864423, 0.12277107057205394, 0.08824957965163749, 0.10027615937559874, 0.10027615937559874, 0.09617966939259756, 0.10918336816375895, 0.09415098222095473, 0.10123394771092674, 0.09208515602186049, 0.10332710672293151, 0.0926464623598867, 0.09937019935938766], [3.0, 53.0, 0.8, 3.7720078707044373, 0.09851141832579778, 0.24853397382384473, 0.08685889638065035, 0.10918336816375895, 0.10173391125761691, 0.09769573467038892, 0.1077130032257564, 0.1063490342699408, 0.09236311002550751, 0.09003632671271142, 0.12426698691192237, 0.08763573312564595, 0.09851141832579778, 0.08867489652961237], [2.0, 79.

In [13]:
# Update cust_item received data
for i in range(n_cust):   # for each customer
    for j in range(n_item):  # for each item
        cust_item[i][j] = cust_item[i][j] + model.x[i,j]()
cust_item

[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0

In [14]:
# declare a concrete model
model = ConcreteModel()

# declare the decision variables
n_cust = len(custdata)  # 1047
n_item = len(itemdata)  # 111

model.x = Var(range(n_cust), range(n_item), domain = Binary)  # Item sent to that customer? (n_cust*n_item)

# i refers to the customer
# j refers to the item

# specify the objective: Maximize the total expected margin minus the cost of the samples for the next round of boxes

# total expected margin: whether item sent * PurchaseProb_Seg of that item * margin if purchase
expectedmargin = sum(model.x[i,j]*iteminfo[j][segnum[i]+4]*iteminfo[j][3] for i in range(n_cust) for j in range(n_item))
# total sample costs: whether item sent * sample cost
samplecosts = sum(model.x[i,j]*iteminfo[j][2] for i in range(n_cust) for j in range(n_item))
model.Objective = Objective(expr = expectedmargin - samplecosts, sense= maximize) 

# Constraint: 5 items need to be sent to each customer
model.FiveItemSent =  ConstraintList() 
for i in range(n_cust): # for each customer
    model.FiveItemSent.add(expr = sum(model.x[i,j] for j in range(n_item)) == 5)

# Constraint: Can’t run out of inventory
model.InventoryLimit =  ConstraintList() 
for j in range(n_item): # for each item
    model.InventoryLimit.add(expr = sum(model.x[i,j] for i in range(n_cust)) <= iteminfo[j][1])

# Constraint: Can’t send a customer a sample they’ve already received
model.NeverRecieved =  ConstraintList() 
for i in range(n_cust): # for each customer
    model.NeverRecieved.add(expr = sum(model.x[i,j]*cust_item[i][j] for j in range(n_item)) == 0)

# specify the solver and solve
opt = SolverFactory('glpk')
results = opt.solve(model)#, tee = True)

In [15]:
solutionprint() # obj: -898.2635839546256

Expected profit (total expected margin - sample costs): -898.2635839546256

The items sent to each customer: 

Customer 0 : send item  4  8  23 31 50  

Customer 1 : send item  13 22 40 58 70  

Customer 2 : send item  1  3  4  24 88  

Customer 3 : send item  23 50 53 80 107 

Customer 4 : send item  37 48 53 64 107 

Customer 5 : send item  13 22 31 43 70  

Customer 6 : send item  12 50 53 100107 

Customer 7 : send item  4  29 37 48 77  

Customer 8 : send item  43 65 92 100110 

Customer 9 : send item  29 44 46 67 88  

Customer 10 : send item  13 50 67 94 108 

Customer 11 : send item  5  23 32 48 104 

Customer 12 : send item  10 37 48 53 64  

Customer 13 : send item  2  29 46 88 100 

Customer 14 : send item  13 22 43 67 70  

Customer 15 : send item  16 43 46 92 110 

Customer 16 : send item  4  12 29 37 77  

Customer 17 : send item  8  23 50 68 88  

Customer 18 : send item  23 50 53 80 107 

Customer 19 : send item  4  29 37 48 77  

Customer 20 : send item  5  23 48 50 10

Customer 316 : send item  3  4  24 44 88  

Customer 317 : send item  4  8  50 68 88  

Customer 318 : send item  5  23 32 48 50  

Customer 319 : send item  8  16 38 41 43  

Customer 320 : send item  8  31 50 68 88  

Customer 321 : send item  33 37 48 64 107 

Customer 322 : send item  5  8  50 58 88  

Customer 323 : send item  4  8  50 68 88  

Customer 324 : send item  10 33 37 64 107 

Customer 325 : send item  29 39 56 79 84  

Customer 326 : send item  2  29 44 46 88  

Customer 327 : send item  4  29 37 48 77  

Customer 328 : send item  22 31 40 43 70  

Customer 329 : send item  7  12 34 39 88  

Customer 330 : send item  13 22 43 67 70  

Customer 331 : send item  46 69 73 96 108 

Customer 332 : send item  7  12 39 87 88  

Customer 333 : send item  8  16 41 43 70  

Customer 334 : send item  29 39 56 79 84  

Customer 335 : send item  46 69 73 96 108 

Customer 336 : send item  5  23 48 50 104 

Customer 337 : send item  46 69 73 96 108 

Customer 338 : send item  13 22 


Customer 565 : send item  1  13 67 94 108 

Customer 566 : send item  46 69 73 96 108 

Customer 567 : send item  6  16 38 100110 

Customer 568 : send item  1  3  24 38 44  

Customer 569 : send item  10 29 39 79 84  

Customer 570 : send item  3  4  24 44 88  

Customer 571 : send item  7  34 39 87 88  

Customer 572 : send item  10 29 56 64 79  

Customer 573 : send item  5  23 32 48 50  

Customer 574 : send item  3  5  23 32 50  

Customer 575 : send item  16 43 46 92 100 

Customer 576 : send item  2  29 44 46 100 

Customer 577 : send item  48 69 73 96 108 

Customer 578 : send item  5  23 32 50 104 

Customer 579 : send item  1  3  4  24 44  

Customer 580 : send item  4  22 40 43 58  

Customer 581 : send item  29 39 56 79 84  

Customer 582 : send item  3  4  24 44 88  

Customer 583 : send item  8  23 50 68 88  

Customer 584 : send item  12 39 70 87 88  

Customer 585 : send item  13 22 43 67 70  

Customer 586 : send item  46 69 73 96 108 

Customer 587 : send item  4  29

Customer 807 : send item  2  44 46 88 100 

Customer 808 : send item  22 40 43 58 70  

Customer 809 : send item  10 29 56 79 84  

Customer 810 : send item  5  23 32 48 50  

Customer 811 : send item  4  29 37 48 77  

Customer 812 : send item  34 39 70 87 88  

Customer 813 : send item  5  23 48 50 104 

Customer 814 : send item  10 16 48 53 64  

Customer 815 : send item  3  5  32 48 50  

Customer 816 : send item  12 34 39 70 87  

Customer 817 : send item  16 38 92 100110 

Customer 818 : send item  5  23 48 50 104 

Customer 819 : send item  6  16 38 43 100 

Customer 820 : send item  3  5  32 50 104 

Customer 821 : send item  13 22 31 43 70  

Customer 822 : send item  2  29 67 88 100 

Customer 823 : send item  21 46 48 69 108 

Customer 824 : send item  1  50 67 94 108 

Customer 825 : send item  10 33 37 48 64  

Customer 826 : send item  1  13 50 67 94  

Customer 827 : send item  29 44 67 88 100 

Customer 828 : send item  29 39 56 79 84  

Customer 829 : send item  1  3  

In [16]:
df_sol2 = solutiontodf()
df_sol2

,Cust #,Item 0,Item 1,Item 2,Item 3,Item 4,Item 5,Item 6,Item 7,Item 8,...,Item 101,Item 102,Item 103,Item 104,Item 105,Item 106,Item 107,Item 108,Item 109,Item 110
0,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1042,1042,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1043,1043,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1044,1044,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1045,1045,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [17]:
# Export df_sol2 to excel
df_sol2.to_excel("P5_BirchboxSolutionPart2_Shih-Yuan_Wang.xlsx", index=False)

### Interpretation of the solution for the next month with zero replenishment

The binary result shows whether the item should be sent to the customer. A "1" indicates that this item sample should be sent to the customer. For example, the sample of the item 4, 8, 23, 31, 50 should be send to Customer 0 next month.

The expected profit (loss) under the optimal solution: $ (898.26358)

Thus, if Birchbox failed to source any new samples next month, the businese might make a loss.